## torch ver

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import csv

### Data reading

In [35]:
# data reading
with open('./data/titanicTrain.csv', 'r') as f:
    data = list(csv.reader((f)))
    data = np.array(data)

In [36]:
data = data[:1000]
data[0, [0, 3, 4, 5, 6, 8]]

array(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare'],
      dtype='<U82')

In [37]:
def data_clean(data, withLabel=True):
    x_train = data[1:, [0, 3, 4, 5, 6, 8]]
    y_train = data[1:, 1]
    pclass = list()
    sex = list()
    embarked = list()
    for i in range(len(x_train)):
        # sex -- class data one hot
        if x_train[i, 1] == 'male':
            sex.append([1, 0])
        else:
            sex.append([0, 1])
        # age
        if x_train[i, 2] == '':
            x_train[i, 2] = '-100'
        # fare
        if x_train[i, 5] == '':
            x_train[i, 5] = '-200'
        # pclass -- class data one hot
        if x_train[i, 0] == '1':
            pclass.append([1, 0, 0])
        elif x_train[i, 0] == '2':
            pclass.append([0, 1, 0])
        else:
            pclass.append([0, 0, 1])
    if withLabel: 
        y_train = np.array(y_train).astype('int64')
    sex = np.array(sex)
    pclass = np.array(pclass)
    x_train = x_train[:, [2, 3, 4, 5]].astype('float32')
    # normalization
    x_train[:, 0] = x_train[:, 0]/100
    x_train[:, 1] = x_train[:, 1]/6
    x_train[:, 2] = x_train[:, 2]/6
    x_train[:, 3] = x_train[:, 2]/200
    # combination of class-data and continuous data
    x_train = np.hstack([pclass, sex, x_train])
    if withLabel: 
        return x_train, y_train
    else:
        return x_train

x_train, y_train = data_clean(data)
x_train.shape, y_train.shape

((999, 9), (999,))

In [38]:
# random splitting of training set and testing set
ind = np.arange(len(x_train))
np.random.shuffle(ind)
ind_train = ind[0:int(len(x_train)*0.8)+1]
ind_test = ind[int(len(x_train)*0.8)+1:]
x_test = x_train[ind_test]
y_test = y_train[ind_test]
x_train = x_train[ind_train]
y_train = y_train[ind_train]
len(x_test), len(x_train)

(199, 800)

In [39]:
# validation data
with open('./data/titanicQuestion.csv', 'r') as f:
    data_valid = list(csv.reader((f)))
    data_valid = np.array(data_valid)

x_valid = data_clean(data_valid, withLabel=False)
x_valid.shape

(309, 9)

In [40]:
from collections import Counter
Counter(data[1:, 1])

Counter({'0': 577, '1': 422})

### hyperparameter

In [41]:
epoch = 2000
l_r = 1e-4
n_in = len(x_train[0])
n_out = 2
p_drop = .2

### network structure

In [42]:
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.hidden1 = nn.Linear(n_in, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, n_out)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.dropout(x, p=p_drop)
        x = F.relu(self.hidden2(x))
        x = F.dropout(x, p=p_drop)
        out = F.softmax(self.hidden3(x))
        return out

In [43]:
model = LR()
loss_func = nn.CrossEntropyLoss()
op = torch.optim.RMSprop(model.parameters(), lr=l_r)

### Training

In [44]:
np.set_printoptions(precision=3)
for epo in range(epoch):
    # convert to variables
    x = Variable(torch.from_numpy(x_train.astype('float32')))
    y = Variable(torch.from_numpy(y_train.astype('int64')))

    # clear gradient w.r.t. parameters 
    op.zero_grad()

    # forward to get output
    prediction = model(x)

    # calculate loss
    loss = loss_func(prediction, y)

    # backward to get gradient
    loss.backward()

    # update parameters
    op.step() 

    if (epo+1)%100 == 0:
        pred_train = np.array([np.argmax(prediction.data.numpy()[i]) for i in range(len(prediction.data.numpy()))])
        accu_train = (len(y_train)-np.sum(np.power(pred_train-y_train, 2)))/len(y_train)
        tt = Variable(torch.from_numpy(x_test.astype('float32')))
        prediction_test = model(tt)
        pred_test = np.array([np.argmax(prediction_test.data.numpy()[i]) for i in range(len(prediction_test.data.numpy()))])
        accu_test = (len(y_test)-np.sum(np.power(pred_test-y_test, 2)))/len(y_test)
        print('Epoch:', epo+1,
              ', LossTrain:', loss.data.numpy(),
              ', AccuTrain:', accu_train,
              ', AccuTest:', accu_test)
    pred = model(Variable(torch.from_numpy(x_valid.astype('float32')))).data.numpy()

Epoch: 100 , LossTrain: [ 0.542] , AccuTrain: 0.79125 , AccuTest: 0.78391959799
Epoch: 200 , LossTrain: [ 0.517] , AccuTrain: 0.7925 , AccuTest: 0.798994974874
Epoch: 300 , LossTrain: [ 0.509] , AccuTrain: 0.80375 , AccuTest: 0.793969849246
Epoch: 400 , LossTrain: [ 0.505] , AccuTrain: 0.79875 , AccuTest: 0.798994974874
Epoch: 500 , LossTrain: [ 0.502] , AccuTrain: 0.8 , AccuTest: 0.819095477387
Epoch: 600 , LossTrain: [ 0.5] , AccuTrain: 0.80375 , AccuTest: 0.814070351759
Epoch: 700 , LossTrain: [ 0.498] , AccuTrain: 0.815 , AccuTest: 0.819095477387
Epoch: 800 , LossTrain: [ 0.493] , AccuTrain: 0.81375 , AccuTest: 0.814070351759
Epoch: 900 , LossTrain: [ 0.488] , AccuTrain: 0.8275 , AccuTest: 0.809045226131
Epoch: 1000 , LossTrain: [ 0.485] , AccuTrain: 0.82875 , AccuTest: 0.804020100503
Epoch: 1100 , LossTrain: [ 0.483] , AccuTrain: 0.82625 , AccuTest: 0.804020100503
Epoch: 1200 , LossTrain: [ 0.481] , AccuTrain: 0.82625 , AccuTest: 0.804020100503
Epoch: 1300 , LossTrain: [ 0.48] , A

In [45]:
from collections import Counter
sur = [np.argmax(pred[i]) for i in range(len(pred))]
Counter(sur), sur

(Counter({0: 257, 1: 52}),
 [0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0

In [77]:
xx == sur

True